In [1]:
import sacrebleu

# Define the reference and hypothesis sentences
reference = ["This is a test sentence."]
hypothesis = "This is a test sentence with a small error."

# Calculate BLEU score
bleu_score = sacrebleu.sentence_bleu(hypothesis, reference)
print(f"BLEU score: {bleu_score.score:.4f}")

# Calculate CHRF++ score
chrfpp_score = sacrebleu.sentence_chrf(hypothesis, reference)
print(f"CHRF++ score: {chrfpp_score.score:.4f}")

BLEU score: 41.1134
CHRF++ score: 81.1120


In [129]:
import copy
import random
import numpy as np
import pandas as pd

from pymystem3 import Mystem
from tqdm.notebook import tqdm
from uralicNLP import uralicApi
from collections import Counter

In [130]:
vocab_1 = pd.read_csv("DICTIONARY_MANS_RUS_new.csv")
vocab_2 = pd.read_csv("DICTIONARY_MANS_RUS_2.csv")

In [131]:
print(len(vocab_2))
vocab_2=vocab_2.dropna(subset=["rus"])
print(len(vocab_2))

5769
5766


In [132]:
vocab_2.loc[1140], vocab_2.loc[1141], vocab_2.loc[1143]

(mans                     кар
 rus     III. длинный предмет
 note                     NaN
 Name: 1140, dtype: object,
 mans         кар
 rus     II. очко
 note         NaN
 Name: 1141, dtype: object,
 mans         кар
 rus     I. дятел
 note         NaN
 Name: 1143, dtype: object)

In [133]:
rus_translate = vocab_2.rus.tolist()
mans_translate = vocab_2.mans.tolist()
note_translate = vocab_2.note.tolist()

In [134]:
# rus_translate = vocab_2.rus.tolist()
for i in range(len(rus_translate)):
    if "гл. прист." in rus_translate[i]:
        # print(i)
        rus_translate[i] = rus_translate[i].replace("гл. прист.", note_translate[i])
        note_translate[i] = ""
    if "гл. приставка" in rus_translate[i]:
        rus_translate[i] = rus_translate[i].replace("гл. приставка", note_translate[i])
        note_translate[i] = ""
    if "гл.прист." in rus_translate[i]:
        # print(i)
        rus_translate[i] = rus_translate[i].replace("гл.прист.", note_translate[i])
        note_translate[i] = ""

In [135]:
for i in range(len(rus_translate)):
    for n in ["III.", "II.", "I.", "I", "II", "III"]:
        # print(i)
        if n in rus_translate[i]:
            # print(i)
            rus_translate[i] = rus_translate[i].replace(n, "")
        if not isinstance(note_translate[i], float):
            # print(note_translate[i])
            if n in note_translate[i]:
                note_translate[i] = note_translate[i].replace(n, "")
    rus_translate[i] = rus_translate[i].strip()
    if not isinstance(note_translate[i], float):
        note_translate[i] = note_translate[i].strip()

In [136]:
bad = ["4)", ") 2)", "),2)", ") ,2)", ") 3)",");2)", "); 2)", "2)", "),1)", ");", ")"]
for i in range(len(note_translate)):
    if isinstance(note_translate[i], float):
        continue
    clean_last = False
    for n in bad:
        if n in note_translate[i]:
            # print(note_translate[i])
            note_translate[i] = note_translate[i].replace(n, ",")
            clean_last = True
    if clean_last:
        # print(note_translate[i])
        res = note_translate[i].split(",")[:-1]
        note_translate[i] = []
        for elem in res:
            if elem != " " or elem != "":
                note_translate[i].append(elem)
        note_translate[i] = (", ".join(note_translate[i])).strip()
        if note_translate[i][-1] == ",":
            note_translate[i] = note_translate[i][:-1]
        # print("FINAL:", note_translate[i])

In [137]:
vocab_2.rus = rus_translate
vocab_2.mans = mans_translate
vocab_2.note = note_translate

In [138]:
rus_translate = vocab_1.rus.tolist()
mans_translate = vocab_1.mans.tolist()
note_translate = vocab_1.note.tolist()

In [139]:
rus_translate[47] = "один"
note_translate[47] = "акв хум - один мужчина"

rus_translate[903] = "ещё"
note_translate[903] = "неа̄сюм иӈыт ёхты - отец ещё не приехал"

vocab_1.rus = rus_translate
vocab_1.mans = mans_translate
vocab_1.note = note_translate

In [158]:
vocab = pd.concat([vocab_1, vocab_2])

In [159]:
dt = pd.read_csv("../dataset/train.csv")
dt.loc[1].target

'Хоса ман вāти минасыт, э̄тэ минэ̄гт, хо̄талэ минэ̄гт акваг.'

In [160]:
rus_translate = vocab.rus.tolist()
mans_translate = vocab.mans.tolist()
note_translate = vocab.note.tolist()

In [161]:
words = {}
for i in range(len(rus_translate)):
    s = f"{mans_translate[i]}@@@{rus_translate[i]}"
    if s not in words:
        words[s] = []
    if isinstance(note_translate[i], float):
        continue
    words[s].append(note_translate[i])

rus_translate = []
mans_translate = []
note_translate = []
for key in words:
    m, r = key.split("@@@")
    mans_translate.append(m)
    rus_translate.append(r)
    if len(words[key]) > 0:
        note_translate.append(", ".join(words[key]))
    else:
        note_translate.append(np.nan)

In [162]:
data = {'mans': mans_translate,
        'rus': rus_translate,
        'note': note_translate}

vocab = pd.DataFrame(data)

In [172]:
vocab_mans_dict = {}
vocab_rus_dict = {}
full_rus_dict = {}
ids = vocab.index
mans = vocab.mans.tolist()
rus = vocab.rus.tolist()
note = vocab.note.tolist()

In [173]:
for i in range(len(ids)):
    rus_word = rus[i]
    mans_word = mans[i]
    if note[i] is not np.nan:
        rus_word += f" ({note[i]})"
    if mans_word not in vocab_mans_dict:
        vocab_mans_dict[mans_word] = []
    if rus_word not in vocab_rus_dict:
        vocab_rus_dict[rus_word] = []

    vocab_mans_dict[mans_word].append(rus_word)
    vocab_rus_dict[rus_word].append(mans_word)
    

In [174]:
for i in range(len(ids)):
    rus_word = rus[i]
    words = rus_word.split()
    if note[i] is not np.nan:
        rus_word += f" ({note[i]})"
    for word in words:
        if word in ["с", "но", "и", "а", "что", "в", "на", "от", "не", "то", "это", "тот", "же", "это"]:
            continue
        if word not in full_rus_dict:
            full_rus_dict[word] = []
        add_word = True
        for item in full_rus_dict[word]:
            if item[0] == rus_word:
                add_word = False
        if add_word:
            full_rus_dict[word].append([rus_word, vocab_rus_dict[rus_word], i])

In [175]:
# full_rus_dict

In [176]:
def replace_symbols(text, f=True):
    for symbol in "!@#$%,.1234567890/?\'\"\\«»_:;_…()”":
        text = text.replace(symbol, " ")
    first_words = set((' '.join(text.lower().split())).split())
    
    for symbol in "!@#$%,.1234567890-/?\'\"\\«»_-:;_…–()”":
        text = text.replace(symbol, " ")
    second_words = set((' '.join(text.lower().split())).split())
    return list(first_words | second_words)
    
def lemmatize_mans_sentence(sentence):
    sentence = replace_symbols(sentence)
    words = []
    for word in sentence:
        norm_form = uralicApi.lemmatize(word, "mns")
        if len(norm_form) == 0:
            words.append(word)
        else:
            words.extend(sorted(uralicApi.lemmatize(word, "mns")))
    return list(set(words) | set(sentence))

system = Mystem()
def lemmatize_rus_sentence(sentence):
    lemmas = system.lemmatize(sentence)
    lemmas_clean = []
    for word in lemmas:
        if word in "!@#$%,.1234567890-/?\'\"\\«»_-:;_…–()”":
            continue
        lemmas_clean.append(word)
    sentence = replace_symbols(sentence)
    return list(set(" ".join(lemmas_clean).split()) | set(sentence))

In [177]:
def get_rus_translate(words, vocab, sep=" | "):
    translate = []
    for word in words:
        if word in vocab:
            translate.append(word + " - " + ", ".join(vocab[word]))
    return sep.join(translate)

In [178]:
i = 1
print(dt.pr_target[i])
print(dt.pr_source[i])
get_rus_translate(lemmatize_mans_sentence(dt.pr_target[i]), vocab_mans_dict)

Хоса ман вāти минасыт, э̄тэ минэ̄гт, хо̄талэ минэ̄гт акваг.
Долго ли, коротко ли едут, всю ночь едут, днём едут.


'э̄т - ночь | акваг - всегда, повсюду, постоянно, навсегда, везде, всюду | ва̄ти - короткий, краткий | хоса - длинный, далёкий, долго, длинный  далёкий | минуӈкве - идти, ехать, лететь, пойти, уходить | хо̄тал - день, солнце'

In [179]:
def get_mans_translate(words, vocab, sep=" | "):
    translate = []
    used = set()
    for word in words:
        if word in vocab:
            for example in vocab[word]:
                if example[2] not in used:
                    translate.append(example[0] + " - " + ", ".join(example[1]))
                    used.add(example[2])
    return sep.join(translate)

In [180]:
i = 1500
print(dt.pr_target[i])
print(dt.pr_source[i])
get_mans_translate(lemmatize_rus_sentence(dt.pr_source[i]), full_rus_dict)

То̄нт тувыл тотум пормасанэ̄н ань тэ̄н суссылтасанэ̄н ос са̄всыр тэ̄лат урыл ма̄навн потыртасы̄г.
Тогда они принесли  экспонаты  и много интересного об этом рассказали


'они идут под звуки горна - Горн рохтан суил минэ̄гыт | они (двое) - тэ̄н | тогда - мо̄лал, то̄нт, тōнт | тогда же - аквтапорат, аквтаэ̄ртын | приносить жертву - йирхатуӈкве | приносить - тыг-тотыгла̄луӈкве | много - са̄в | у нас много учёных - ма̄н ма̄вт са̄в учё̄ный ōлы | рассказывать сказки - мо̄йтуӈкве | рассказывать - потырта̄лыглаӈкве, потыртаӈкве | говорить  беседовать, разговаривать  рассказывать - потыртаӈкве | интересный - осьхолиӈ, пӯмасьӈ, пӯмасиӈ | интерес  шутка;  интересный  смешной - осхоль | интересный  странный  шутливый - осьхолыӈ'

In [181]:
vocab_rus_dict["взять"]

['виӈкве',
 'хот-виӈкве',
 'хот-няртумтаӈкве',
 'хот-виӈкве (ам хот-вы̄гум, наӈ хот-вы̄гын, нэ̄н хот-выгы̄н, на̄н хот-выгы̄н)']

In [162]:
full_rus_dict["взять"]

[['взять', ['виӈкве', 'хот-виӈкве'], 469],
 ['взять (с собой)', ['ёт-виӈкве'], 845]]

In [92]:
text = "Мама-мылаm, раму каждый вечер перед сном."
lemmatize_rus_sentence(text)

['раму',
 'сном',
 'мылаm',
 'рама',
 'сон',
 'мама-мылаm',
 'вечер',
 'перед',
 'мама',
 ',',
 'каждый']

In [109]:
lemmatize_mans_sentence(dt.pr_target[0]), dt.pr_source[0], dt.pr_target[0]

(['ке', 'о̄лыс', 'о̄луӈкве', 'пуӈк', 'пуӈке', 'мēкыс'],
 'Если голова его ослабнет:',
 'Пуӈке ке мēкыс о̄лыс:')

In [93]:
lemmatize_mans_sentence(dt.pr_target[1]), dt.pr_target[1]

(['ва̄ти',
  'вāти',
  'хо̄талэ',
  'хоса',
  'э̄т',
  'ман',
  'акваг',
  'минуӈкве',
  'минэ̄гт',
  'минасыт',
  'хо̄тал',
  'э̄тэ'],
 'Хоса ман вāти минасыт, э̄тэ минэ̄гт, хо̄талэ минэ̄гт акваг.')

In [94]:
lemmatize_mans_sentence(dt.pr_target[1])

['ва̄ти',
 'вāти',
 'хо̄талэ',
 'хоса',
 'э̄т',
 'ман',
 'акваг',
 'минуӈкве',
 'минэ̄гт',
 'минасыт',
 'хо̄тал',
 'э̄тэ']

In [101]:
lemmatize_mans_sentence(dt.pr_target[1])[0][1], lemmatize_mans_sentence(dt.pr_target[1])[1][1]

('а', 'ā')

In [67]:
i = 3
lemmatize_mans_sentence(dt.pr_target[i]), dt.pr_target[i]

((['а̄гиӈпы̄г', 'кол', 'а̄гиӈ', 'ёмас', 'пы̄г', 'пыгыӈ'],
  ['а̄гиӈ-пыгыӈ', 'кол', 'а̄гиӈ', 'ёмас', 'пыгыӈ']),
 'А̄гиӈ-пыгыӈ ёмас кол.')

In [68]:
from pymystem3 import Mystem

m = Mystem()
text = "Мама-мыла раму каждый вечер перед сном."
lemmas = m.lemmatize(text)
print(lemmas)

['мама', '-', 'мыть', ' ', 'рама', ' ', 'каждый', ' ', 'вечер', ' ', 'перед', ' ', 'сон', '.', '\n']


In [69]:
replace_symbols(text)

['раму', 'мыла', 'сном', 'вечер', 'мама', 'мама-мыла', 'перед', 'каждый']